Hàm này là để chuyển pixel trong ROI 2D thành điểm 3D

Hàm phía dưới là xử lý với 1 ảnh

In [3]:
# Chuyển từ pixel trong ROI 2D thành điểm 3D trong hệ tọa độ camera 
# -> tạo point cloud ROI

# project_to_3d.py

import numpy as np
import open3d as o3d

class ROIProjector3D:
    def __init__(self, intrinsics):
        """
        intrinsics = {
            "fx": ...,
            "fy": ...,
            "cx": ...,
            "cy": ...
        }
        """
        self.fx = intrinsics["fx"]
        self.fy = intrinsics["fy"]
        self.cx = intrinsics["cx"]
        self.cy = intrinsics["cy"]

    def project_roi_to_pointcloud(self, rgb_img, depth_img, roi_bbox, depth_scale=0.001):
        """
        Chuyển ROI 2D → Point Cloud 3D.
        @param rgb_img: numpy array ảnh RGB (H, W, 3)
        @param depth_img: numpy array depth (H, W)
        @param roi_bbox: (x, y, w, h)
        @param depth_scale: scale từ mm → m
        @return: Open3D PointCloud chứa điểm trong ROI
        """
        x, y, w, h = roi_bbox

        points = []
        colors = []

        for v in range(y, y + h):
            for u in range(x, x + w):
                Z = depth_img[v, u] * depth_scale
                if Z <= 0:
                    continue

                X = (u - self.cx) * Z / self.fx
                Y = (v - self.cy) * Z / self.fy

                points.append([X, Y, Z])
                colors.append(rgb_img[v, u] / 255.0)

        if len(points) == 0:
            print("⚠️ Không có điểm nào trong ROI có depth hợp lệ.")
            return None

        pc_roi = o3d.geometry.PointCloud()
        pc_roi.points = o3d.utility.Vector3dVector(np.array(points))
        pc_roi.colors = o3d.utility.Vector3dVector(np.array(colors))

        # 🔁 Lật point cloud theo trục Y để khớp hướng ảnh RGB
        pc_roi.transform([[1, 0, 0, 0],
                        [0, -1, 0, 0],
                        [0, 0, 1, 0],
                        [0, 0, 0, 1]])

        return pc_roi

Test thử với 1 ảnh

In [ ]:
# import cv2
# intrinsics = {"fx": 643.90087890625, "fy": 643.1365356445312, "cx": 650.2113037109375, "cy": 355.79559326171875}  
# roi_bbox = (560, 150, 300, 330)

# rgb = cv2.imread("Public data/Public data train/rgb/0002.png")
# depth = cv2.imread("Public data/Public data train/depth/0002.png", cv2.IMREAD_UNCHANGED)

# projector = ROIProjector3D(intrinsics)
# pc = projector.project_roi_to_pointcloud(rgb, depth, roi_bbox)

# if pc:
#     o3d.io.write_point_cloud("output/roi_0002.ply", pc)
#     print("✅ ROI PLY saved: output/roi_0002.ply")

[Open3D WARNING] Write PLY failed: unable to open file: output\roi_0002.ply
✅ ROI PLY saved: output/roi_0002.ply


Xử lý trên cả tập data

In [7]:
import os
import cv2

In [9]:
# =====================
# Cấu hình
# =====================
rgb_folder = "./Public data/Public data train/rgb"
depth_folder = "./Public data/Public data train/depth"
output_folder = "./data/roi_pointclouds"
os.makedirs(output_folder, exist_ok=True)

intrinsics = {
    "fx": 643.90087890625, 
    "fy": 643.1365356445312, 
    "cx": 650.2113037109375, 
    "cy": 355.79559326171875
}

roi_bbox = (560, 150, 300, 330)

projector = ROIProjector3D(intrinsics)

# =====================
# Lặp qua tất cả cặp ảnh RGB + depth
# =====================
for filename in os.listdir(rgb_folder):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):
        rgb_path = os.path.join(rgb_folder, filename)
        depth_path = os.path.join(depth_folder, filename)  # giả sử cùng tên

        rgb_img = cv2.imread(rgb_path)
        depth_img = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)

        if rgb_img is None or depth_img is None:
            print(f"⚠️ Không tìm thấy RGB hoặc depth cho {filename}")
            continue

        pc_roi = projector.project_roi_to_pointcloud(rgb_img, depth_img, roi_bbox)
        if pc_roi is None:
            continue

        output_path = os.path.join(output_folder, filename.replace(".jpg", ".ply").replace(".png", ".ply"))
        o3d.io.write_point_cloud(output_path, pc_roi, write_ascii=True)
        print(f"Saved ROI point cloud to {output_path}")

Saved ROI point cloud to ./data/roi_pointclouds\0000.ply
Saved ROI point cloud to ./data/roi_pointclouds\0001.ply
Saved ROI point cloud to ./data/roi_pointclouds\0002.ply
Saved ROI point cloud to ./data/roi_pointclouds\0003.ply
Saved ROI point cloud to ./data/roi_pointclouds\0004.ply
Saved ROI point cloud to ./data/roi_pointclouds\0005.ply
Saved ROI point cloud to ./data/roi_pointclouds\0006.ply
Saved ROI point cloud to ./data/roi_pointclouds\0007.ply
Saved ROI point cloud to ./data/roi_pointclouds\0008.ply
Saved ROI point cloud to ./data/roi_pointclouds\0009.ply
Saved ROI point cloud to ./data/roi_pointclouds\0010.ply
Saved ROI point cloud to ./data/roi_pointclouds\0011.ply
Saved ROI point cloud to ./data/roi_pointclouds\0012.ply
Saved ROI point cloud to ./data/roi_pointclouds\0013.ply
Saved ROI point cloud to ./data/roi_pointclouds\0014.ply
Saved ROI point cloud to ./data/roi_pointclouds\0015.ply
Saved ROI point cloud to ./data/roi_pointclouds\0016.ply
Saved ROI point cloud to ./data

In [14]:
import open3d as o3d
pc = o3d.io.read_point_cloud("D:/AI_project/VAR/data/roi_pointclouds/0002.ply")
print(pc)
o3d.visualization.draw_geometries([pc])


PointCloud with 95860 points.
